# Implementing the cropping function

## The various sub-tasks involved are :

* Get shape of the big image : (p,q)
* Define the size of the tile : (a,b) = (50,50)
* Determine the number of useful tiles; ensure all tiles are of equal size (a,b) and discard the ones that arent - Might lose a few due to boundary conditions but thats okay!
* Carry out Cropping of the tiles sequentially! 
* Obtain the vertex of the each of the cropped tiles.
* Save each of these cropped tiles in respective folder.

In [1]:
#Import Stuff 
import os
import cv2
import math
import numpy as np

In [2]:
#Read the image
#source_path = '/Users/keshavaprasad/Desktop/image/MG48_3day_bs/testingcrop/S051_00.tif'
#source_path = '/Users/keshavaprasad/Desktop/image/S25/S25_00.tif'
source_path = '//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/S51_Pia_Only.tif'
image=cv2.imread(source_path)

In [3]:
# Image Basics
print(image.shape)
# Get Shape
p = image.shape[0]# p
q = image.shape[1]#q
print("Height:",p,"Width:",q)

# Defining the dimensions of a tile 
a = b = 50

#************************
cropped_tiles = [] #list for storing all cropped tile images
vertices = [] #list for storing lists of vertex values # Defining vertices of tiles for cropping
ctc = []#list for storing lists of tile numbers
#cropped_tile_list = []
mean_values = []# list for storing mean of the pixel values of each cropped tile 
background_cropped_tiles = [] #list for storing background tile images
# Cropping image Sequentially
i = j = cropped_tile_counter = correct_tile_counter = wrong_dim_tile_counter = mean= correct_dim_tile_counter= 0


(9536, 10340, 3)
Height: 9536 Width: 10340


In [4]:
# Carry oout the sequential cropping store data into respective lists
for i in range(0,math.floor(p),a):#range(start, stop, step) # to get the resolution right!
    for j in range(0,math.floor(q),b):
        #Defining the cropping box
        #tile = image[j:j+b,i:i+a]#y1: y2, x1: x2
        y1 = j
        x1 = i
        y2 = j+b
        x2 = i+a
        
        tile = image[x1:x2,y1:y2]# making crops of each subimage and then storing in tile variable and then appended to cropped_tiles list
        vertex = [x1,x2,y1,y2] # list of vertex values of each tiles
        cropped_tile_counter +=1
        
        # Calculating mean for each tile and
        mean = np.mean(tile)
        
        # checking for tiles with wrong dimensions
        if tile.shape[0]!=50 or tile.shape[1]!= 50:
            wrong_dim_tile_counter += 1
        
        # Keeping only tiles with correct dimensions # Append the vertex for each of these tiles into vertices list 
        # Append the correct_tile_counter variable into ctc list
        if tile.shape[0] == 50 and tile.shape[1] == 50:
            correct_dim_tile_counter +=1
            if mean>15:# imposing the mean value filter in order to remove lot of unnecessary background
                correct_tile_counter +=1
                #cropped_tiles.append(tile)
                cropped_tiles.append([correct_tile_counter,tile])
                ctc.append(correct_tile_counter)
                #print("correct_tile_counter:",correct_tile_counter,"with vertex:",vertex)
                vertices.append(vertex)
                # appending these values into mean_values list
                mean_values.append(mean)                        
### THE TWO KEY TAKEAWAYS:number of croppedtiles and respective vertex of each cropped tile

In [5]:
print(len(cropped_tiles))

3689


In [6]:
import pandas as pd

# Calling DataFrame constructor after zipping # both lists, with columns specified 
#cropped_data = pd.DataFrame(list(zip(ctc, vertices)),columns =['Tile_Counter', 'Vertices']) 
cropped_data = pd.DataFrame(list(zip(ctc, vertices,mean_values)),columns =['Tile_Counter', 'Vertices','mean_values']) 
#cropped_data

In [8]:
cropped_data.head()

,Tile_Counter,Vertices,mean_values
0,1,"[4450, 4500, 7550, 7600]",15.3204
1,2,"[4450, 4500, 7600, 7650]",15.3148
2,3,"[4450, 4500, 7650, 7700]",15.0784
3,4,"[4500, 4550, 7550, 7600]",16.5260
4,5,"[4500, 4550, 7600, 7650]",17.1000


In [9]:
#cropped_data.loc[cropped_data['Tile_Counter'] == 21555] #searching rows based on specific values
cropped_data.loc[cropped_data['mean_values'] > 15] 

,Tile_Counter,Vertices,mean_values
0,1,"[4450, 4500, 7550, 7600]",15.3204
1,2,"[4450, 4500, 7600, 7650]",15.3148
2,3,"[4450, 4500, 7650, 7700]",15.0784
3,4,"[4500, 4550, 7550, 7600]",16.5260
4,5,"[4500, 4550, 7600, 7650]",17.1000
...,...,...,...
3684,3685,"[8550, 8600, 4950, 5000]",17.1160
3685,3686,"[8550, 8600, 5000, 5050]",16.7952
3686,3687,"[8550, 8600, 5050, 5100]",16.8088
3687,3688,"[8600, 8650, 4950, 5000]",15.7060


In [10]:
import pickle
with open('global_xy_MG48_3day_bs_S051.pkl', 'wb') as g:
    pickle.dump(cropped_data, g)

In [11]:
print("Total number of tiles:",cropped_tile_counter)
print("Wasted tiles due boundary conditions:",(wrong_dim_tile_counter/cropped_tile_counter)*100,"%")
print("Useless background tiles:",(correct_dim_tile_counter - correct_tile_counter)/cropped_tile_counter*100,"%")
print("Useful tiles:",(correct_tile_counter/cropped_tile_counter)*100,"%")


Total number of tiles: 39537
Wasted tiles due boundary conditions: 1.0041227204896679 %
Useless background tiles: 89.66537673571591 %
Useful tiles: 9.33050054379442 %


In [12]:
print(len(cropped_tiles))
print(cropped_tiles[1000][0])

3689
1001


## Saving useful tiles that are sequentially cropped and are of correct dimensions and qualify mean value filter

In [13]:
# Saving the tiles in a respective folder

Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/S051/allcroppedtiles/'
#Dest_Path = '/Users/keshavaprasad/Desktop/image/S25/'
for k in range(len(cropped_tiles)):
        pass
        #print("K:",k)
        #cv2.imwrite(str(k)+ ".tif", tiles[k])
        cv2.imwrite(os.path.join(Dest_Path,str(k)+".tif"), cropped_tiles[k][1])